<a href="https://colab.research.google.com/github/rajivsam/interactive_tutorials/blob/master/notebooks/Graph_Retail_Descriptive_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

%%capture
!git clone https://github.com/rajivsam/interactive_tutorials.git
!rsync -av interactive_tutorials/notebooks/data  interactive_tutorials/notebooks/img  interactive_tutorials/notebooks/tools ./ --exclude=.git
!pip install python-arango
!pip install arangopipe==0.0.6.9.5
!pip install pandas PyYAML==5.1.1 sklearn2
!pip install jsonpickle
!pip3 install networkx
!pip3 install matplotlib
!pip3 install adbnx-adapter

In [2]:
from arangopipe.arangopipe_storage.arangopipe_api import ArangoPipe
from arangopipe.arangopipe_storage.arangopipe_admin_api import ArangoPipeAdmin
from arangopipe.arangopipe_storage.arangopipe_config import ArangoPipeConfig
from arangopipe.arangopipe_storage.managed_service_conn_parameters import ManagedServiceConnParam
mdb_config = ArangoPipeConfig()
msc = ManagedServiceConnParam()
conn_params = { msc.DB_SERVICE_HOST : "arangoml.arangodb.cloud", \
                        msc.DB_SERVICE_END_POINT : "createDB",\
                        msc.DB_SERVICE_NAME : "createDB",\
                        msc.DB_SERVICE_PORT : 8529,\
                        msc.DB_CONN_PROTOCOL : 'https',\
                        msc.DB_REPLICATION_FACTOR: 3}
        
mdb_config = mdb_config.create_connection_config(conn_params)
admin = ArangoPipeAdmin(reuse_connection = False, config = mdb_config)
ap_config = admin.get_config()
ap = ArangoPipe(config = ap_config)
proj_info = {"name": "Retail_Graph_Analytics"}
proj_reg = admin.register_project(proj_info)
mdb_config.get_cfg()

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


API endpoint: https://arangoml.arangodb.cloud:8529/_db/_system/createDB/createDB


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'arangoml.arangodb.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Host Connection: https://arangoml.arangodb.cloud:8529


2021-06-07 12:23:27,710 - arangopipe_admin_logger - ERROR - Error connecting to DB, trying again...


API endpoint: https://arangoml.arangodb.cloud:8529/_db/_system/createDB/createDB


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'arangoml.arangodb.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Host Connection: https://arangoml.arangodb.cloud:8529


2021-06-07 12:23:57,025 - arangopipe_admin_logger - ERROR - Obtained connection with retry!


{'arangodb': {'DB_end_point': 'createDB',
  'DB_service_host': 'arangoml.arangodb.cloud',
  'DB_service_name': 'createDB',
  'DB_service_port': 8529,
  'arangodb_replication_factor': 3,
  'conn_protocol': 'https',
  'dbName': 'MLhkdtob1ade72igvk844rpc',
  'password': 'MLsqvffcuiibgv7l5nqlium',
  'username': 'ML5atpzpgp4sgbi30ot2f4a'},
 'mlgraph': {'graphname': 'enterprise_ml_graph'}}

In [3]:
conn = mdb_config.get_cfg()
login = conn['arangodb']

In [4]:
login

{'DB_end_point': 'createDB',
 'DB_service_host': 'arangoml.arangodb.cloud',
 'DB_service_name': 'createDB',
 'DB_service_port': 8529,
 'arangodb_replication_factor': 3,
 'conn_protocol': 'https',
 'dbName': 'MLhkdtob1ade72igvk844rpc',
 'password': 'MLsqvffcuiibgv7l5nqlium',
 'username': 'ML5atpzpgp4sgbi30ot2f4a'}

In [5]:
!chmod -R 755 ./tools/*

In [6]:
# Restore an ArangoDB dump into the database.
!./tools/arangorestore  -c none --server.endpoint  http+ssl://{login["DB_service_host"]}:{login["DB_service_port"]} --server.username {login["username"]} \
--server.database {login["dbName"]} \
--server.password {login["password"]} \
--replication-factor 3  \
--input-directory "data/retail_freq_cust_data_dump"

2021-06-07T12:23:58Z [195] INFO [05c30] {restore} Connected to ArangoDB 'http+ssl://arangoml.arangodb.cloud:8529'
2021-06-07T12:23:59Z [195] INFO [abeb4] {restore} Database name in source dump is 'MLqugvfcgt448hq4bpqccnw'
2021-06-07T12:23:59Z [195] INFO [9b414] {restore} # Re-creating document collection 'Customers'...
2021-06-07T12:24:00Z [195] INFO [9b414] {restore} # Re-creating document collection 'datasets'...
2021-06-07T12:24:01Z [195] INFO [9b414] {restore} # Re-creating document collection 'deployment'...
2021-06-07T12:24:02Z [195] INFO [9b414] {restore} # Re-creating document collection 'devperf'...
2021-06-07T12:24:03Z [195] INFO [9b414] {restore} # Re-creating document collection 'featuresets'...
2021-06-07T12:24:04Z [195] INFO [9b414] {restore} # Re-creating document collection 'Items'...
2021-06-07T12:24:04Z [195] INFO [9b414] {restore} # Re-creating document collection 'modelparams'...
2021-06-07T12:24:05Z [195] INFO [9b414] {restore} # Re-creating document collection 'mo

## Methodology Overview

This notebook provides the descriptive features of the frequent customer group. These features are obtained by applying graph analytic concepts to the frequent customer group. To do so, we need to represent the frequent customer group as a graph. In particular, this group is modeled as a bi-partite graph with the customer nodes and item nodes forming the two disjoint vertex sets. The purchase of an item by a customer as part of an invoice is modeled as an edge.

[link text](im)
This notebook will use ideas from:

1.    [Statistical Analysis of Network Data](https://bigdata.unl.edu/documents/ASA_Workshop_Materials/Tutorial%20Statistical%20Analysis%20of%20Network%20Data.pdf): Provides a methodology for characterizing network structure
2.   [Basic Notions for the Analysis of Affiliate Networks](https://github.com/arangoml/networkx-adapter/blob/master/bipartite_clustering.pdf) : Provides a methodology for analyzing bi-partite graphs

The descriptive characteristics of the graph are captured by:


1.   Vertex and Edge Characteristics
2.   Network Cohesion

These perspectives are important because:


1.   They provide insights about customer behavior or the concept modeled by the 
2.   The provide the basis for developing other machine learning tasks for the problem. For example, we may be interested in the following:
*    How can we develop a generative model for the graph 
*   How do we model the formation of links in the network

These tasks tell us how customers make purchases. For details of developing a generative model on bipartite graphs, see [Bipartite Graphs as Models of Complex Networks](http://denif.ens-lyon.fr/data/algorithmique_reseaux_telecoms_enslyon/2007/cours/biparti.pdf) and [Measuring and Modeling Bipartite Graphs with Community Structure](https://www.osti.gov/servlets/purl/1561802). Developing these models requires the descriptive characterization of the graph, which is what is done in this notebook. This chararacterization provides information about properties such as degree distribution and clustering characteristics of nodes which are used to develop the models discussed.

While summary statistics and distributions can be used to provide one descriptive perspective of the network (vertex and edge characteristics), information about grouping characteristics provides the complimentary perspective (network-cohesion). There are multiple ways to describe network-cohension. In this notebook, we will describe two ways to capture or describe cohesive tendencies in the notebook. These are the number of connected components in the graph and measurements describing clique formation in the graph.

The rest of the notebook will provide the implementation of the methodology discussed here. There is one other conceptual idea that we require in the toolbox for the analysis of bi-partite graphs. This is the notion of a _projected graph_










In [15]:
import networkx as nx
from adbnx_adapter.arangoDB_networkx_adapter import ArangoDB_Networkx_Adapter

# Specify the connection to the ArangoDB Database
con = {'dbName': login["dbName"],
 'username': login["username"],
 'password': login["password"],
 'hostname': login["DB_service_host"],
 'port': login["DB_service_port"]}

# Create Adapter instance
ma = ArangoDB_Networkx_Adapter(conn = con)

freq_customers = {'vertexCollections': {'Customers': {},
                                         'Items': {}},
                   'edgeCollections': {'Purchases': {'_from', '_to', 'spend'}}}

# Export networkX graph                                  
g = ma.create_networkx_graph(graph_name = '',  graph_attributes = freq_customers)


In [16]:
gp = g.to_undirected()

In [9]:
cust_nodes = [n for n in g.nodes() if n.startswith("Customers")]
item_nodes = [n for n in g.nodes() if n.startswith("Items")]

In [10]:
len(cust_nodes)

447

In [11]:
len(item_nodes)

75

In [12]:
len(g.edges())

4475

In [13]:
gp.is_directed()

False